# Backtesting Analysis Deep Dive - 

This notebook provides comprehensive analysis of portfolio backtesting, including:
- Walk-forward optimization
- Transaction cost analysis
- Rebalancing frequency optimization
 - Monte Carlo simulation for strategy robustness
- Optimal rebalancing frequency analysis
- Strategy stability over time

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from src.data.fetcher import DataFetcher
from src.optimization.mean_variance import MeanVarianceOptimizer
from src.backtesting.engine import BacktestEngine, BacktestConfig, RebalanceFrequency

# Suppress verbose logging from all modules
import logging

# Set logging level for optimization module
logging.getLogger('src.optimization.mean_variance').setLevel(logging.WARNING)
logging.getLogger('src.optimization').setLevel(logging.WARNING)

# Also suppress any root logger messages
logging.basicConfig(level=logging.WARNING)

plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Data and Setup

In [ ]:
# Load historical data
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'JPM']
fetcher = DataFetcher()

# Fetch longer history for robust backtesting
prices = fetcher.fetch_price_data(
    tickers=tickers,
    start_date='2015-01-01',
    end_date='2024-01-01'
)

print(f"Data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")

# Calculate returns
returns = prices.pct_change().dropna()

In [ ]:
# Cell 2.5: Load ML Predictions from ML Notebook

import json
import pandas as pd
import os

print("="*70)
print("LOADING ML PREDICTIONS")
print("="*70)

# Check if ML results exist
ml_results_path = '../data/ml_results/ml_predictions.json'

if os.path.exists(ml_results_path):
    # Load ML predictions
    with open(ml_results_path, 'r') as f:
        ml_results = json.load(f)
    
    print("✓ Successfully loaded ML predictions")
    print(f"  Created: {ml_results['metadata']['created_date']}")
    print(f"  Data end date: {ml_results['metadata']['data_end_date']}")
    print(f"  Blend ratio: {ml_results['metadata']['blend_ratio']}")
    
    # Extract key components
    ml_expected_returns = ml_results['ml_expected_returns']
    ml_optimal_weights = ml_results['optimal_weights']['ml_enhanced']
    ml_blend_ratio = ml_results['metadata']['blend_ratio']
    
    # Display ML predictions
    print("\nML Expected Returns (Annualized):")
    for ticker, ret in ml_expected_returns.items():
        historical = ml_results['historical_baseline'].get(ticker, 0)
        print(f"  {ticker}: {ret:.1%} (Historical: {historical:.1%})")
    
    # Performance improvement from ML
    print(f"\nML Performance Improvement:")
    print(f"  Traditional Sharpe: {ml_results['model_performance']['traditional_sharpe']:.3f}")
    print(f"  ML-Enhanced Sharpe: {ml_results['model_performance']['ml_enhanced_sharpe']:.3f}")
    print(f"  Improvement: {ml_results['model_performance']['improvement_pct']:.1f}%")
    
    # Walk-forward validation stats
    print(f"\nWalk-Forward Validation:")
    print(f"  Periods tested: {ml_results['walk_forward_stats']['periods_tested']}")
    print(f"  Success rate: {ml_results['walk_forward_stats']['success_rate']*100:.0f}%")
    print(f"  Average Sharpe: {ml_results['walk_forward_stats']['average_sharpe']:.3f}")
    
else:
    print("⚠️  ML predictions not found!")
    print(f"   Expected location: {ml_results_path}")
    print("   Please run the ML notebook (06_ml_price_prediction.ipynb) first.")
    print("\n   Using default values for demonstration...")
    
    # Fallback to reasonable defaults if ML results don't exist
    ml_expected_returns = {
        'AAPL': 0.15,
        'MSFT': 0.12,
        'GOOGL': 0.14,
        'AMZN': 0.16,
        'JPM': 0.10
    }
    ml_blend_ratio = 0.6
    ml_optimal_weights = {ticker: 0.2 for ticker in tickers}

# Verify tickers match
ml_tickers = set(ml_expected_returns.keys())
data_tickers = set(tickers)

if ml_tickers != data_tickers:
    print(f"\n⚠️  Warning: Ticker mismatch!")
    print(f"   ML tickers: {ml_tickers}")
    print(f"   Data tickers: {data_tickers}")
    
    # Handle GOOG vs GOOGL mismatch if present
    if 'GOOG' in data_tickers and 'GOOGL' in ml_tickers:
        print("   Mapping GOOGL predictions to GOOG...")
        ml_expected_returns['GOOG'] = ml_expected_returns.get('GOOGL', 0.12)
        if 'GOOG' not in ml_optimal_weights and 'GOOGL' in ml_optimal_weights:
            ml_optimal_weights['GOOG'] = ml_optimal_weights['GOOGL']

print("\n" + "="*70)

In [ ]:
# Cell 2.6: Debug ML Data Loading

print("="*70)
print("DEBUGGING ML DATA LOADING")
print("="*70)

# Check if the file exists
import os
ml_results_path = '../data/ml_results/ml_predictions.json'
print(f"1. Checking file existence:")
print(f"   Path: {ml_results_path}")
print(f"   Exists: {os.path.exists(ml_results_path)}")

if os.path.exists(ml_results_path):
    # Check file contents
    print(f"\n2. File info:")
    print(f"   Size: {os.path.getsize(ml_results_path)} bytes")
    print(f"   Modified: {pd.Timestamp.fromtimestamp(os.path.getmtime(ml_results_path))}")
    
    # Load and display raw contents
    import json
    with open(ml_results_path, 'r') as f:
        raw_data = json.load(f)
    
    print(f"\n3. File contents - ML predictions:")
    if 'ml_expected_returns' in raw_data:
        for ticker, value in raw_data['ml_expected_returns'].items():
            print(f"   {ticker}: {value:.4f} ({value*100:.1f}%)")
    else:
        print("   ERROR: 'ml_expected_returns' key not found!")
        print("   Available keys:", list(raw_data.keys()))
    
    print(f"\n4. Currently loaded values:")
    for ticker, value in ml_expected_returns.items():
        print(f"   {ticker}: {value:.4f} ({value*100:.1f}%)")
        
else:
    print("\n⚠️  FILE NOT FOUND! Using fallback values.")
    print("   This explains why you're seeing hardcoded values!")
    print("\n   Next steps:")
    print("   1. Run the ML notebook (06_ml_price_prediction.ipynb)")
    print("   2. Make sure to run the export cell (Cell 10)")
    print("   3. Check that the file is created in ../data/ml_results/")

## 2. Walk-Forward Optimization

In [ ]:
def walk_forward_backtest(
    prices, 
    lookback_period=252,  # 1 year
    rebalance_frequency='monthly',
    optimization_method='max_sharpe'
):
    """
    Perform walk-forward backtesting
    """
    results = []
    portfolio_values = [100000]  # Starting capital
    weights_history = []
    
    # Set up rebalancing dates
    if rebalance_frequency == 'monthly':
        rebalance_dates = pd.date_range(
            start=prices.index[lookback_period],
            end=prices.index[-1],
            freq='MS'  # Month start
        )
    elif rebalance_frequency == 'quarterly':
        rebalance_dates = pd.date_range(
            start=prices.index[lookback_period],
            end=prices.index[-1],
            freq='QS'  # Quarter start
        )
    
    optimizer = MeanVarianceOptimizer()
    current_weights = None
    
    for i, date in enumerate(prices.index[lookback_period:]):
        # Check if we need to rebalance
        if date in rebalance_dates:
            # Get historical data for optimization
            historical_prices = prices.loc[:date].tail(lookback_period)
            
            # Optimize portfolio
            result = optimizer.optimize(
                historical_prices,
                objective=optimization_method
            )
            
            current_weights = pd.Series(
                result.weights,
                index=result.asset_names
            )
            weights_history.append({
                'date': date,
                'weights': current_weights.to_dict()
            })
        
        # Calculate portfolio value
        if current_weights is not None:
            # Calculate returns
            if i > 0:
                daily_returns = prices.loc[date] / prices.iloc[lookback_period + i - 1] - 1
                portfolio_return = (current_weights * daily_returns).sum()
                portfolio_values.append(portfolio_values[-1] * (1 + portfolio_return))
            
            results.append({
                'date': date,
                'portfolio_value': portfolio_values[-1] if i > 0 else portfolio_values[0],
                'weights': current_weights.to_dict()
            })
    
    return pd.DataFrame(results), pd.DataFrame(weights_history)

# Run walk-forward backtest
print("Running walk-forward backtest...")
backtest_results, weights_history = walk_forward_backtest(
    prices,
    lookback_period=252,
    rebalance_frequency='monthly',
    optimization_method='max_sharpe'
)

# Calculate performance metrics
final_value = backtest_results['portfolio_value'].iloc[-1]
total_return = (final_value / 100000 - 1) * 100
years = len(backtest_results) / 252
annual_return = (final_value / 100000) ** (1/years) - 1

print(f"\nWalk-Forward Backtest Results:")
print(f"Final Portfolio Value: ${final_value:,.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Annualized Return: {annual_return:.2%}")

## 3. Transaction Cost Analysis

In [ ]:
def backtest_with_costs(prices, initial_weights, rebalance_freq, transaction_cost):
    """
    Run backtest with different transaction costs
    """
    import logging
    original_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)
    config = BacktestConfig(
        initial_capital=100000,
        transaction_cost_pct=transaction_cost,
        rebalance_frequency=rebalance_freq
    )
    
    engine = BacktestEngine(config)
    
    # Create strategy that returns fixed weights
    def fixed_weight_strategy(historical_prices):
        return initial_weights
    
    results = engine.run_backtest(prices, fixed_weight_strategy)
    return results

# Test different transaction costs
transaction_costs = [0, 0.001, 0.002, 0.005, 0.01]  # 0 to 1%
rebalance_frequencies = [
    RebalanceFrequency.DAILY,
    RebalanceFrequency.WEEKLY,
    RebalanceFrequency.MONTHLY,
    RebalanceFrequency.QUARTERLY
]

# Use equal weights for comparison
equal_weights = np.array([0.2] * 5)

# Store results
cost_analysis_results = {}

for freq in rebalance_frequencies:
    cost_analysis_results[freq.value] = {}
    
    for cost in transaction_costs:
        results = backtest_with_costs(
            prices.tail(252*3),  # Last 2 years
            equal_weights,
            freq,
            cost
        )
        
        final_value = results.portfolio_values.iloc[-1]
        total_return = (final_value / 100000 - 1)
        
        cost_analysis_results[freq.value][cost] = {
            'final_value': final_value,
            'total_return': total_return,
            'total_costs': results.total_transaction_costs + results.total_slippage_costs,
            'turnover': results.turnover_rate
        }

# Create visualization
fig = go.Figure()

for freq in rebalance_frequencies:
    returns = [cost_analysis_results[freq.value][cost]['total_return'] * 100 
               for cost in transaction_costs]
    
    fig.add_trace(go.Scatter(
        x=[c * 100 for c in transaction_costs],
        y=returns,
        mode='lines+markers',
        name=freq.value.capitalize(),
        line=dict(width=2)
    ))

fig.update_layout(
    title='Impact of Transaction Costs on Returns by Rebalancing Frequency',
    xaxis_title='Transaction Cost (%)',
    yaxis_title='Total Return (%)',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()

# Turnover analysis
print("\nPortfolio Turnover by Rebalancing Frequency:")
for freq in rebalance_frequencies:
    turnover = cost_analysis_results[freq.value][0.001]['turnover']
    print(f"{freq.value.capitalize()}: {turnover:.1%} annual turnover")

## 4. Comparing Strategies Across Market Conditions

In [ ]:
# Define market periods
market_periods = {
    'Bull Market (2017-2019)': ('2017-01-01', '2019-12-31'),
    'COVID Crash (2020)': ('2020-01-01', '2020-12-31'),
    'Recovery (2021-2022)': ('2021-01-01', '2022-12-31'),
    'Recent (2023)': ('2023-01-01', '2023-12-31')
}

# Strategies to test
strategies_to_test = {
    'max_sharpe': 'Max Sharpe',
    'min_volatility': 'Min Volatility'
}

# Results storage
period_results = {}

for period_name, (start, end) in market_periods.items():
    period_results[period_name] = {}
    
    # Get data for this period
    period_prices = prices[start:end]
    
    if len(period_prices) < 20:  # Skip if too few data points
        continue
    
    # Optimize at the beginning of the period
    train_end = period_prices.index[0] - timedelta(days=1)
    train_start = train_end - timedelta(days=365)
    train_prices = prices[train_start:train_end]
    
    optimizer = MeanVarianceOptimizer()
    
    for strategy_key, strategy_name in strategies_to_test.items():
        # Optimize
        result = optimizer.optimize(train_prices, objective=strategy_key)
        weights = pd.Series(result.weights, index=result.asset_names)
        
        # Calculate performance
        period_returns = period_prices.pct_change().dropna()
        portfolio_returns = (period_returns * weights).sum(axis=1)
        
        cumulative_return = (1 + portfolio_returns).cumprod()
        total_return = cumulative_return.iloc[-1] - 1
        volatility = portfolio_returns.std() * np.sqrt(252)
        sharpe = (portfolio_returns.mean() * 252 - 0.02) / volatility
        
        # Calculate max drawdown
        running_max = cumulative_return.expanding().max()
        drawdown = (cumulative_return - running_max) / running_max
        max_drawdown = drawdown.min()
        
        period_results[period_name][strategy_name] = {
            'return': total_return,
            'volatility': volatility,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'cumulative': cumulative_return
        }

# Create comparison visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=list(market_periods.keys()),
    shared_yaxes=True
)

colors = {'Max Sharpe': 'blue', 'Min Volatility': 'green'}

for i, (period_name, results) in enumerate(period_results.items()):
    row = i // 2 + 1
    col = i % 2 + 1
    
    for strategy_name, metrics in results.items():
        fig.add_trace(
            go.Scatter(
                x=metrics['cumulative'].index,
                y=(metrics['cumulative'] - 1) * 100,
                mode='lines',
                name=strategy_name,
                line=dict(color=colors[strategy_name], width=2),
                showlegend=(i == 0)  # Only show legend once
            ),
            row=row, col=col
        )

fig.update_yaxes(title_text='Cumulative Return (%)', row=1, col=1)
fig.update_yaxes(title_text='Cumulative Return (%)', row=2, col=1)
fig.update_layout(
    height=800,
    title_text='Strategy Performance Across Market Conditions',
    template='plotly_white'
)

fig.show()

# Summary table
print("\nPerformance Summary by Period:")
print("=" * 80)

for period_name, results in period_results.items():
    print(f"\n{period_name}:")
    for strategy_name, metrics in results.items():
        print(f"  {strategy_name}:")
        print(f"    Return: {metrics['return']:.2%}")
        print(f"    Volatility: {metrics['volatility']:.2%}")
        print(f"    Sharpe: {metrics['sharpe']:.3f}")
        print(f"    Max Drawdown: {metrics['max_drawdown']:.2%}")

## 5. Monte Carlo Simulation for Strategy Robustness## 5. Monte Carlo Simulation for Strategy Robustness

In [ ]:
# Cell 6: Monte Carlo Simulation for Strategy Robustness

# Define the ML strategy creation function here so it's available for Monte Carlo
def create_ml_enhanced_strategy(ml_predictions, ml_blend_ratio=0.6):
    """
    Creates a strategy function that uses loaded ML predictions
    blended with historical returns
    """
    def ml_enhanced_strategy(historical_prices):
        """
        Strategy that combines ML predictions with dynamic historical data
        """
        from scipy.optimize import minimize
        
        # Calculate historical statistics from the provided data
        returns = historical_prices.pct_change().dropna()
        historical_mean = returns.mean() * 252
        historical_cov = returns.cov() * 252
        
        # Convert ML predictions to Series aligned with historical data
        ml_returns = pd.Series(
            index=historical_prices.columns,
            dtype=float
        )
        
        # Map ML predictions to the correct tickers
        for ticker in historical_prices.columns:
            if ticker in ml_predictions:
                ml_returns[ticker] = ml_predictions[ticker]
            elif ticker == 'GOOG' and 'GOOGL' in ml_predictions:
                ml_returns[ticker] = ml_predictions['GOOGL']
            else:
                ml_returns[ticker] = historical_mean[ticker]
        
        # Blend ML predictions with historical returns
        blended_returns = ml_blend_ratio * ml_returns + (1 - ml_blend_ratio) * historical_mean
        
        # Optimize portfolio with blended returns
        n_assets = len(blended_returns)
        
        def objective(w):
            port_return = np.dot(w, blended_returns)
            port_vol = np.sqrt(np.dot(w.T, np.dot(historical_cov.values, w)))
            sharpe = (port_return - 0.02) / port_vol
            return -sharpe  # Negative for minimization
        
        # Constraints and bounds
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = tuple((0.05, 0.40) for _ in range(n_assets))
        x0 = np.ones(n_assets) / n_assets
        
        # Optimize
        result = minimize(objective, x0, method='SLSQP', 
                         bounds=bounds, constraints=constraints,
                         options={'ftol': 1e-9, 'disp': False})
        
        if not result.success:
            return x0  # Return equal weights if optimization fails
        
        return result.x
    
    return ml_enhanced_strategy

# Now continue with the Monte Carlo simulation
def monte_carlo_backtest(prices, n_simulations=1000, test_period_days=252, 
                        ml_predictions=None, ml_blend_ratio=0.6):
    """
    Run Monte Carlo simulations to test strategy robustness
    Now includes ML-enhanced strategy testing
    """
    results = []
    
    # Create ML strategy if predictions provided
    if ml_predictions is not None:
        ml_strategy = create_ml_enhanced_strategy(ml_predictions, ml_blend_ratio)
    
    for sim in range(n_simulations):
        # Randomly select training period
        max_start = len(prices) - test_period_days - 252  # Need 1 year for training
        start_idx = np.random.randint(252, max_start)
        
        # Split data
        train_prices = prices.iloc[start_idx-252:start_idx]
        test_prices = prices.iloc[start_idx:start_idx+test_period_days]
        
        # Test both traditional and ML strategies
        strategies = {
            'traditional': 'max_sharpe',
            'ml_enhanced': ml_strategy if ml_predictions is not None else None
        }
        
        for strategy_name, strategy in strategies.items():
            if strategy is None:
                continue
                
            if strategy_name == 'traditional':
                # Traditional optimization
                optimizer = MeanVarianceOptimizer()
                result = optimizer.optimize(train_prices, objective=strategy)
                weights = pd.Series(result.weights, index=result.asset_names)
            else:
                # ML-enhanced strategy
                weights = pd.Series(strategy(train_prices), index=train_prices.columns)
            
            # Test on out-of-sample data
            test_returns = test_prices.pct_change().dropna()
            portfolio_returns = (test_returns * weights).sum(axis=1)
            
            # Calculate metrics
            total_return = (1 + portfolio_returns).prod() - 1
            annual_return = (1 + total_return) ** (252/len(portfolio_returns)) - 1
            volatility = portfolio_returns.std() * np.sqrt(252)
            sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
            
            # Calculate max drawdown
            cum_returns = (1 + portfolio_returns).cumprod()
            running_max = cum_returns.expanding().max()
            drawdown = (cum_returns - running_max) / running_max
            max_drawdown = drawdown.min()
            
            results.append({
                'simulation': sim,
                'strategy': strategy_name,
                'train_start': prices.index[start_idx-252],
                'test_start': prices.index[start_idx],
                'annual_return': annual_return,
                'volatility': volatility,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_drawdown,
                'total_return': total_return
            })
    
    return pd.DataFrame(results)

# Run Monte Carlo simulation with ML predictions
print("Running Monte Carlo simulation with ML strategies (this may take a minute)...")
mc_results = monte_carlo_backtest(
    prices, 
    n_simulations=500,  # Reduced for faster execution
    ml_predictions=ml_expected_returns,
    ml_blend_ratio=ml_blend_ratio
)

# Separate results by strategy
traditional_results = mc_results[mc_results['strategy'] == 'traditional']
ml_results = mc_results[mc_results['strategy'] == 'ml_enhanced']

# Create comparative visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Sharpe Ratio Distribution', 'Annual Return Distribution', 
                   'Risk-Return Scatter (All Simulations)', 'Drawdown Comparison')
)

# 1. Sharpe ratio distributions
fig.add_trace(
    go.Histogram(x=traditional_results['sharpe_ratio'], 
                 name='Traditional', opacity=0.7,
                 marker_color='blue', nbinsx=30),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=ml_results['sharpe_ratio'], 
                 name='ML-Enhanced', opacity=0.7,
                 marker_color='green', nbinsx=30),
    row=1, col=1
)

# 2. Return distributions
fig.add_trace(
    go.Histogram(x=traditional_results['annual_return'], 
                 name='Traditional', opacity=0.7,
                 marker_color='blue', nbinsx=30, showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=ml_results['annual_return'], 
                 name='ML-Enhanced', opacity=0.7,
                 marker_color='green', nbinsx=30, showlegend=False),
    row=1, col=2
)

# 3. Risk-return scatter
fig.add_trace(
    go.Scatter(
        x=traditional_results['volatility'],
        y=traditional_results['annual_return'],
        mode='markers',
        marker=dict(size=5, opacity=0.5, color='blue'),
        name='Traditional'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
        x=ml_results['volatility'],
        y=ml_results['annual_return'],
        mode='markers',
        marker=dict(size=5, opacity=0.5, color='green'),
        name='ML-Enhanced'
    ),
    row=2, col=1
)

# 4. Drawdown comparison
fig.add_trace(
    go.Box(y=traditional_results['max_drawdown'], 
           name='Traditional', marker_color='blue'),
    row=2, col=2
)
fig.add_trace(
    go.Box(y=ml_results['max_drawdown'], 
           name='ML-Enhanced', marker_color='green'),
    row=2, col=2
)

# Update axes
fig.update_xaxes(tickformat='.0%', row=2, col=1)
fig.update_yaxes(tickformat='.0%', row=2, col=1)
fig.update_yaxes(tickformat='.0%', row=2, col=2)

fig.update_layout(height=800, title_text='Monte Carlo Simulation: ML vs Traditional (500 runs)')
fig.show()

# Summary statistics comparison
print("\nMonte Carlo Simulation Summary:")
print("="*70)

# Calculate comparative statistics
comparison_stats = pd.DataFrame({
    'Traditional': [
        traditional_results['annual_return'].mean(),
        traditional_results['volatility'].mean(),
        traditional_results['sharpe_ratio'].mean(),
        traditional_results['max_drawdown'].mean(),
        traditional_results['sharpe_ratio'].quantile(0.05),
        (traditional_results['sharpe_ratio'] > 0).mean()
    ],
    'ML-Enhanced': [
        ml_results['annual_return'].mean(),
        ml_results['volatility'].mean(),
        ml_results['sharpe_ratio'].mean(),
        ml_results['max_drawdown'].mean(),
        ml_results['sharpe_ratio'].quantile(0.05),
        (ml_results['sharpe_ratio'] > 0).mean()
    ]
}, index=['Avg Annual Return', 'Avg Volatility', 'Avg Sharpe Ratio', 
          'Avg Max Drawdown', '5% VaR Sharpe', 'P(Sharpe > 0)'])

# Format the comparison
for col in comparison_stats.columns:
    for idx in comparison_stats.index:
        if 'Return' in idx or 'Volatility' in idx or 'Drawdown' in idx or 'P(' in idx:
            comparison_stats.loc[idx, col] = f"{comparison_stats.loc[idx, col]:.2%}"
        else:
            comparison_stats.loc[idx, col] = f"{comparison_stats.loc[idx, col]:.3f}"

print(comparison_stats)

# Calculate win rate
ml_wins = 0
for sim in range(len(traditional_results)):
    if ml_results.iloc[sim]['sharpe_ratio'] > traditional_results.iloc[sim]['sharpe_ratio']:
        ml_wins += 1

win_rate = ml_wins / len(traditional_results)
print(f"\nML Win Rate: {win_rate:.1%} (ML outperforms Traditional)")
print(f"Average Sharpe Improvement: {(ml_results['sharpe_ratio'].mean() - traditional_results['sharpe_ratio'].mean()):.3f}")

# Statistical significance test
from scipy import stats
t_stat, p_value = stats.ttest_rel(ml_results['sharpe_ratio'], traditional_results['sharpe_ratio'])
print(f"\nPaired t-test: t={t_stat:.3f}, p-value={p_value:.4f}")
if p_value < 0.05:
    print("✓ ML improvement is statistically significant at 5% level")
else:
    print("✗ ML improvement is not statistically significant at 5% level")

## 6. Optimal Rebalancing Frequency Analysis

In [ ]:
import logging
logging.getLogger('src.optimization.mean_variance').setLevel(logging.ERROR)
logging.getLogger('src.backtesting.engine').setLevel(logging.WARNING)

# Test different rebalancing frequencies with realistic costs
rebalance_analysis = {}
transaction_cost = 0.001  # 0.1% realistic cost

frequencies_to_test = [
    ('Never', RebalanceFrequency.NEVER),
    ('Yearly', RebalanceFrequency.YEARLY),
    ('Quarterly', RebalanceFrequency.QUARTERLY),
    ('Monthly', RebalanceFrequency.MONTHLY),
    ('Weekly', RebalanceFrequency.WEEKLY),
    ('Daily', RebalanceFrequency.DAILY)
]

# Get optimal weights from recent data
recent_prices = prices.tail(252)
optimizer = MeanVarianceOptimizer()
result = optimizer.optimize(recent_prices, objective='max_sharpe')
optimal_weights = result.weights

# Test each frequency
test_prices = prices.tail(252*3)  # Last 3 years

for freq_name, freq_enum in frequencies_to_test:
    config = BacktestConfig(
        initial_capital=100000,
        transaction_cost_pct=transaction_cost,
        rebalance_frequency=freq_enum
    )
    
    engine = BacktestEngine(config)
    
    # Strategy that returns optimal weights
    def optimal_weight_strategy(historical_prices):
        return optimal_weights
    
    results = engine.run_backtest(test_prices, optimal_weight_strategy)
    
    # Calculate metrics
    returns = results.returns
    annual_return = returns.mean() * 252
    volatility = returns.std() * np.sqrt(252)
    sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
    
    rebalance_analysis[freq_name] = {
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe,
        'total_costs': results.total_transaction_costs,
        'turnover': results.turnover_rate,
        'final_value': results.portfolio_values.iloc[-1],
        'trades': len(results.trades)
    }

# Create comparison chart
rebalance_df = pd.DataFrame(rebalance_analysis).T

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Net Returns vs Costs', 'Sharpe Ratio', 'Turnover Rate', 'Number of Trades'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Net returns
fig.add_trace(
    go.Bar(x=rebalance_df.index, y=rebalance_df['annual_return'] * 100, name='Annual Return'),
    row=1, col=1
)

# Sharpe ratio
fig.add_trace(
    go.Bar(x=rebalance_df.index, y=rebalance_df['sharpe_ratio'], name='Sharpe Ratio'),
    row=1, col=2
)

# Turnover
fig.add_trace(
    go.Bar(x=rebalance_df.index, y=rebalance_df['turnover'] * 100, name='Turnover %'),
    row=2, col=1
)

# Number of trades
fig.add_trace(
    go.Bar(x=rebalance_df.index, y=rebalance_df['trades'], name='Trades'),
    row=2, col=2
)

fig.update_yaxes(title_text='Annual Return (%)', row=1, col=1)
fig.update_yaxes(title_text='Sharpe Ratio', row=1, col=2)
fig.update_yaxes(title_text='Annual Turnover (%)', row=2, col=1)
fig.update_yaxes(title_text='Number of Trades', row=2, col=2)

fig.update_layout(height=800, showlegend=False, 
                  title_text='Rebalancing Frequency Analysis (with 0.1% transaction costs)')
fig.show()

# Print detailed analysis
print("\nRebalancing Frequency Analysis:")
print("=" * 80)
print(f"{'Frequency':<15} {'Return':<10} {'Sharpe':<10} {'Costs':<10} {'Turnover':<10} {'Final Value':<15}")
print("=" * 80)

for freq, metrics in rebalance_analysis.items():
    print(f"{freq:<15} {metrics['annual_return']*100:>8.2f}% {metrics['sharpe_ratio']:>9.3f} "
          f"${metrics['total_costs']:>8.0f} {metrics['turnover']*100:>8.1f}% ${metrics['final_value']:>14,.2f}")

## 7. Strategy Stability Analysis

In [ ]:
# Analyze how stable the optimal weights are over time
def analyze_weight_stability(prices, lookback=252, rebalance_freq='monthly'):
    """
    Track how portfolio weights change over time
    """
    optimizer = MeanVarianceOptimizer()
    weight_history = []
    
    # Generate rebalance dates
    if rebalance_freq == 'monthly':
        dates = pd.date_range(start=prices.index[lookback], end=prices.index[-1], freq='MS')
    else:
        dates = pd.date_range(start=prices.index[lookback], end=prices.index[-1], freq='QS')
    
    for date in dates:
        # Get historical data
        # Find the closest date in the index
        hist_end_idx = prices.index.get_indexer([date], method='nearest')[0]
        hist_start_idx = hist_end_idx - lookback
        
        if hist_start_idx < 0:
            continue
            
        hist_prices = prices.iloc[hist_start_idx:hist_end_idx]
        
        # Optimize
        result = optimizer.optimize(hist_prices, objective='max_sharpe')
        
        weight_dict = {'date': date}
        for asset, weight in zip(result.asset_names, result.weights):
            weight_dict[asset] = weight
            
        weight_history.append(weight_dict)
    
    return pd.DataFrame(weight_history).set_index('date')

# Analyze weight stability
weight_history = analyze_weight_stability(prices, rebalance_freq='monthly')

# Visualize weight evolution
fig = go.Figure()

for column in weight_history.columns:
    fig.add_trace(go.Scatter(
        x=weight_history.index,
        y=weight_history[column] * 100,
        mode='lines',
        name=column,
        stackgroup='one'
    ))

fig.update_layout(
    title='Portfolio Weight Evolution Over Time',
    xaxis_title='Date',
    yaxis_title='Weight (%)',
    hovermode='x unified',
    height=500,
    template='plotly_white'
)

fig.show()

# Calculate weight stability metrics
weight_changes = weight_history.diff().abs()
avg_change = weight_changes.mean()
max_change = weight_changes.max()

print("\nWeight Stability Analysis:")
print("=" * 50)
print("Average Monthly Weight Change:")
for asset in avg_change.index:
    print(f"  {asset}: {avg_change[asset]*100:.2f}%")
    
print("\nMaximum Single-Month Weight Change:")
for asset in max_change.index:
    print(f"  {asset}: {max_change[asset]*100:.2f}%")
    
print(f"\nTotal portfolio turnover per rebalance: {weight_changes.sum(axis=1).mean()*100:.2f}%")

## 8. Weight Stability Analysis

In [ ]:
# Cell 8: ML-Enhanced Strategy Backtesting

print("="*70)
print("ML-ENHANCED STRATEGY BACKTESTING")
print("="*70)

# Import necessary optimization functions at module level
from scipy.optimize import minimize
import pandas as pd
import numpy as np

def create_ml_enhanced_strategy(ml_predictions, ml_blend_ratio=0.6):
    """
    Creates a strategy function that uses loaded ML predictions
    blended with historical returns
    """
    def ml_enhanced_strategy(historical_prices):
        """
        Strategy that combines ML predictions with dynamic historical data
        """
        # Calculate historical statistics from the provided data
        returns = historical_prices.pct_change().dropna()
        historical_mean = returns.mean() * 252
        historical_cov = returns.cov() * 252
        
        # Convert ML predictions to Series aligned with historical data
        ml_returns = pd.Series(
            index=historical_prices.columns,
            dtype=float
        )
        
        # Map ML predictions to the correct tickers
        for ticker in historical_prices.columns:
            if ticker in ml_predictions:
                ml_returns[ticker] = ml_predictions[ticker]
            elif ticker == 'GOOG' and 'GOOGL' in ml_predictions:
                ml_returns[ticker] = ml_predictions['GOOGL']
            else:
                print(f"Warning: No ML prediction for {ticker}, using historical")
                ml_returns[ticker] = historical_mean[ticker]
        
        # Blend ML predictions with historical returns
        blended_returns = ml_blend_ratio * ml_returns + (1 - ml_blend_ratio) * historical_mean
        
        # Optimize portfolio with blended returns
        n_assets = len(blended_returns)
        
        def objective(w):
            port_return = np.dot(w, blended_returns)
            port_vol = np.sqrt(np.dot(w.T, np.dot(historical_cov.values, w)))
            sharpe = (port_return - 0.02) / port_vol
            return -sharpe  # Negative for minimization
        
        # Constraints and bounds
        constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
        bounds = tuple((0.05, 0.40) for _ in range(n_assets))
        x0 = np.ones(n_assets) / n_assets
        
        # Optimize
        result = minimize(objective, x0, method='SLSQP', 
                         bounds=bounds, constraints=constraints,
                         options={'ftol': 1e-9, 'disp': False})
        
        if not result.success:
            print(f"Optimization warning: {result.message}")
            return x0  # Return equal weights if optimization fails
        
        return result.x
    
    return ml_enhanced_strategy

# Create strategy using loaded ML predictions
print(f"\nUsing ML predictions loaded from: {ml_results_path}")
print(f"ML blend ratio: {ml_blend_ratio}")

# Define strategies to compare (EXCLUDING Equal Weight)
strategies_to_compare = {
    'Traditional Max Sharpe': lambda prices: MeanVarianceOptimizer().optimize(
        prices, objective='max_sharpe').weights,
    
    'ML-Enhanced (Loaded)': create_ml_enhanced_strategy(
        ml_expected_returns, 
        ml_blend_ratio=ml_blend_ratio
    ),
    
    'ML-Enhanced (Conservative)': create_ml_enhanced_strategy(
        ml_expected_returns,
        ml_blend_ratio=0.4  # More conservative: 40% ML, 60% historical
    ),
    
    'ML-Enhanced (Aggressive)': create_ml_enhanced_strategy(
        ml_expected_returns,
        ml_blend_ratio=0.8  # More aggressive: 80% ML, 20% historical
    )
}

# Run backtests with realistic transaction costs
transaction_cost = 0.001  # 10 basis points
backtest_results = {}

# Use last 3 years of data for backtesting
test_data = prices.tail(252 * 3)

print(f"\nBacktesting period: {test_data.index[0].strftime('%Y-%m-%d')} to {test_data.index[-1].strftime('%Y-%m-%d')}")
print(f"Transaction cost: {transaction_cost*100:.1f}%")
print(f"\nRunning backtests...")

for strategy_name, strategy_func in strategies_to_compare.items():
    print(f"\n  Testing {strategy_name}...", end='')
    
    config = BacktestConfig(
        initial_capital=100000,
        transaction_cost_pct=transaction_cost,
        rebalance_frequency=RebalanceFrequency.MONTHLY
    )
    
    engine = BacktestEngine(config)
    
    try:
        results = engine.run_backtest(test_data, strategy_func)
        
        # Calculate key metrics
        returns = results.returns
        annual_return = returns.mean() * 252
        volatility = returns.std() * np.sqrt(252)
        sharpe = (annual_return - 0.02) / volatility if volatility > 0 else 0
        
        # Calculate max drawdown
        cum_returns = (1 + returns).cumprod()
        running_max = cum_returns.expanding().max()
        drawdown = (cum_returns - running_max) / running_max
        max_drawdown = drawdown.min()
        
        backtest_results[strategy_name] = {
            'annual_return': annual_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'total_costs': results.total_transaction_costs,
            'final_value': results.portfolio_values.iloc[-1],
            'cumulative_returns': cum_returns
        }
        
        print(f" ✓ Sharpe: {sharpe:.3f}")
        
    except Exception as e:
        print(f" ✗ Failed: {str(e)}")
        backtest_results[strategy_name] = {
            'annual_return': np.nan,
            'volatility': np.nan,
            'sharpe_ratio': np.nan,
            'max_drawdown': np.nan,
            'total_costs': np.nan,
            'final_value': np.nan,
            'cumulative_returns': pd.Series()
        }

# Create comprehensive comparison visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Cumulative Returns', 'Risk-Return Profile', 
                   'Sharpe Ratios', 'Max Drawdown'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Define colors for each strategy
colors = {
    'Traditional Max Sharpe': 'blue',
    'ML-Enhanced (Loaded)': 'green',
    'ML-Enhanced (Conservative)': 'orange',
    'ML-Enhanced (Aggressive)': 'red'
}

# 1. Cumulative returns
for strategy_name, metrics in backtest_results.items():
    if len(metrics['cumulative_returns']) > 0:
        fig.add_trace(
            go.Scatter(
                x=metrics['cumulative_returns'].index,
                y=(metrics['cumulative_returns'] - 1) * 100,
                mode='lines',
                name=strategy_name,
                line=dict(width=2, color=colors.get(strategy_name))
            ),
            row=1, col=1
        )

# 2. Risk-return scatter (WITHOUT text labels)
strategy_names = list(backtest_results.keys())
returns = [backtest_results[s]['annual_return'] * 100 for s in strategy_names]
volatilities = [backtest_results[s]['volatility'] * 100 for s in strategy_names]
sharpes = [backtest_results[s]['sharpe_ratio'] for s in strategy_names]

fig.add_trace(
    go.Scatter(
        x=volatilities,
        y=returns,
        mode='markers',  # Removed 'text' from mode
        marker=dict(
            size=15, 
            color=[colors.get(s, 'gray') for s in strategy_names],
            line=dict(width=2, color='black')
        ),
        showlegend=False
    ),
    row=1, col=2
)

# 3. Sharpe ratios comparison
fig.add_trace(
    go.Bar(
        x=strategy_names,
        y=sharpes,
        marker_color=[colors.get(s, 'gray') for s in strategy_names],
        showlegend=False
    ),
    row=2, col=1
)

# 4. Max Drawdown
max_drawdowns = [backtest_results[s]['max_drawdown'] * 100 for s in strategy_names]
fig.add_trace(
    go.Bar(
        x=strategy_names,
        y=max_drawdowns,
        marker_color=[colors.get(s, 'gray') for s in strategy_names],
        showlegend=False
    ),
    row=2, col=2
)

# Update axes
fig.update_xaxes(title_text="", row=2, col=1)
fig.update_xaxes(title_text="", row=2, col=2)
fig.update_yaxes(title_text="Cumulative Return (%)", row=1, col=1)
fig.update_yaxes(title_text="Annual Return (%)", row=1, col=2)
fig.update_xaxes(title_text="Annual Volatility (%)", row=1, col=2)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
fig.update_yaxes(title_text="Max Drawdown (%)", row=2, col=2)

fig.update_layout(
    height=1000, 
    showlegend=True, 
    title_text='ML-Enhanced vs Traditional Strategy Comparison<br>(Using Loaded ML Predictions)',
    margin=dict(b=150)
)
fig.show()

# Print detailed comparison
print("\n" + "="*80)
print("STRATEGY COMPARISON SUMMARY")
print("="*80)
print(f"{'Strategy':<30} {'Return':<10} {'Vol':<10} {'Sharpe':<10} {'MaxDD':<10} {'Costs':<10}")
print("="*80)

for strategy, metrics in backtest_results.items():
    if not np.isnan(metrics['annual_return']):
        print(f"{strategy:<30} "
              f"{metrics['annual_return']*100:>8.1f}% "
              f"{metrics['volatility']*100:>8.1f}% "
              f"{metrics['sharpe_ratio']:>9.2f} "
              f"{metrics['max_drawdown']*100:>8.1f}% "
              f"${metrics['total_costs']:>8.0f}")

# Compare to ML notebook results (with error handling)
print("\n" + "="*80)
print("VALIDATION AGAINST ML NOTEBOOK")
print("="*80)

# Check what keys are actually available in ml_results
if 'model_performance' in ml_results:
    print(f"ML Notebook reported improvement: {ml_results['model_performance']['improvement_pct']:.1f}%")
else:
    print("Note: 'model_performance' key not found in ML results")
    print("Available keys:", list(ml_results.keys()))
    
    # Try to calculate improvement from available data
    if 'walk_forward_stats' in ml_results:
        print(f"ML Notebook walk-forward avg Sharpe: {ml_results['walk_forward_stats']['average_sharpe']:.3f}")

# Calculate backtest improvement
ml_enhanced_sharpe = backtest_results['ML-Enhanced (Loaded)']['sharpe_ratio']
traditional_sharpe = backtest_results['Traditional Max Sharpe']['sharpe_ratio']
backtest_improvement = (ml_enhanced_sharpe - traditional_sharpe) / traditional_sharpe * 100

print(f"\nBacktest Results:")
print(f"Traditional Sharpe: {traditional_sharpe:.3f}")
print(f"ML-Enhanced Sharpe: {ml_enhanced_sharpe:.3f}")
print(f"Improvement: {backtest_improvement:.1f}%")

# Performance breakdown
print("\n" + "="*80)
print("ML STRATEGY SENSITIVITY ANALYSIS")
print("="*80)
conservative_sharpe = backtest_results['ML-Enhanced (Conservative)']['sharpe_ratio']
aggressive_sharpe = backtest_results['ML-Enhanced (Aggressive)']['sharpe_ratio']

print(f"Conservative (40% ML): Sharpe = {conservative_sharpe:.3f}")
print(f"Standard (60% ML):     Sharpe = {ml_enhanced_sharpe:.3f}")
print(f"Aggressive (80% ML):   Sharpe = {aggressive_sharpe:.3f}")
print(f"\nOptimal blend appears to be around {ml_blend_ratio*100:.0f}% ML")

In [ ]:
# Note: Market regime analysis requires period-specific ML model training
# which is computationally intensive. The walk-forward validation in the 
# ML notebook (06_ml_price_prediction.ipynb) already demonstrates 
# out-of-sample performance across different time periods.

## Key Takeaways from Backtesting Analysis

### 1. **Transaction Costs Matter**
- Even small transaction costs (0.1%) significantly impact returns
- Daily rebalancing is almost never optimal due to costs
- Monthly or quarterly rebalancing typically provides the best balance

### 2. **Strategy Performance Varies by Market Regime**
- Max Sharpe performs well in trending markets
- Min Volatility shines during market stress
- No single strategy dominates in all conditions

### 3. **Monte Carlo Results Show Robustness**
- Positive expected returns across most scenarios
- Risk of significant drawdowns exists (~20-30%)
- Sharpe ratios are generally positive but variable

### 4. **Optimal Rebalancing Frequency**
- Monthly or quarterly rebalancing is usually optimal
- Must balance tracking error vs transaction costs
- Consider market volatility when choosing frequency

### 5. **Weight Stability**
- Optimal weights can change significantly month-to-month
- Consider using weight bounds or smoothing
- More stable strategies may have lower turnover costs

## Next Steps
- Implement regime detection to switch strategies
- Test more sophisticated rebalancing rules
- Add stop-loss or drawdown controls
- Consider factor-based approaches